<a href="https://colab.research.google.com/github/tylerle29/Memotron/blob/main/MEMoTron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://huggingface.co/nvidia/Llama-3.1-Nemotron-Nano-4B-v1.1

Cloning into 'Llama-3.1-Nemotron-Nano-4B-v1.1'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 85 (delta 7), reused 0 (delta 0), pack-reused 70 (from 1)
Unpacking objects: 100% (85/85), 84.87 KiB | 2.83 MiB/s, done.
Filtering content: 100% (3/3), 4.42 GiB | 97.42 MiB/s, done.
Encountered 2 file(s) that may not have been copied correctly on Windows:
	model-00002-of-00002.safetensors
	model-00001-of-00002.safetensors

See: `git lfs help smudge` for more details.


In [2]:
!pip install -qU vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.2/438.2 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 138.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB

In [53]:
!nohup python -m vllm.entrypoints.openai.api_server \
  --model Llama-3.1-Nemotron-Nano-4B-v1.1 \
  --trust-remote-code \
  --seed 1 \
  --host "0.0.0.0" \
  --port 5000 \
  --served-model-name "Llama-Nemotron-Nano-4B-v1.1" \
  --tensor-parallel-size 1 \
  --max-model-len 50_000 \
  --gpu-memory-utilization 0.95 \
  --enforce-eager \
  --enable-auto-tool-choice \
  --tool-parser-plugin "Llama-3.1-Nemotron-Nano-4B-v1.1/llama_nemotron_nano_toolcall_parser.py" \
  --tool-call-parser "llama_nemotron_json" \
  --chat-template "Llama-3.1-Nemotron-Nano-4B-v1.1/llama_nemotron_nano_generic_tool_calling.jinja" \
  > vllm.log &

nohup: redirecting stderr to stdout


In [54]:
!while ! grep -q "Application startup complete" vllm.log; do tail -n 1 vllm.log; sleep 5; done

(APIServer pid=30324) INFO:     Application shutdown complete.
(APIServer pid=30324) INFO:     Application shutdown complete.
(APIServer pid=30324) INFO:     Application shutdown complete.
(APIServer pid=30324) INFO:     Application shutdown complete.
(APIServer pid=30324) INFO:     Application shutdown complete.
(APIServer pid=30324) INFO:     Application shutdown complete.
(APIServer pid=30324) INFO:     Application shutdown complete.
(APIServer pid=30324) INFO:     Application shutdown complete.
(APIServer pid=30324) INFO:     Application shutdown complete.


In [55]:
!pip install flask-cors
!pip install flask pyngrok

In [56]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import quote
import re

def kym_search_image(query):
    url = f"https://knowyourmeme.com/search?q={quote(query)}"
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(r.text, "html.parser")

    # 1. find the first meme-page link that contains an image with data-image
    candidates = soup.select('a[href^="/memes/"]')
    page = None

    for a in candidates:
        img = a.select_one("img[data-image]")
        if img:
            page = a
            img_tag = img
            break

    if page is None:
        print("No match")
        return None

    # page URL
    page_url = "https://knowyourmeme.com" + page["href"]

    # high-quality image from data-image
    img_url = img_tag["data-image"]

    print("Page:", page_url)
    print("Image:", img_url)
    return (page_url, img_url)

def scrape_text_kym(url):

    # 1. Request webpage

    headers = {"User-Agent": "Mozilla/5.0"}

    response = requests.get(url, headers=headers)

    response.raise_for_status()



    # 2. Parse HTML

    soup = BeautifulSoup(response.text, "html.parser")

    # Sets up the sections we need to parse.

    sections = {}

    relevant_titles = [

    "About", "Origin", "Spread", "History", "Background", "Meaning", "Usage", "Development", "Overview", "External References"

    ]



    # Parses them.

    for h2 in soup.find_all("h2"):

        title = h2.get_text(strip=True)

        if title in relevant_titles:

            content = []



            # Walk through siblings until the next h2

            for sib in h2.next_siblings:

                if getattr(sib, "name", None) == "h2":

                    break

                if sib.name in ["p"]:

                    text = sib.get_text(" ", strip=True)

                    if text:
                        text = re.sub(r"\[.*?\]", "", text)
                        content.append(text.strip())



            sections[title] = "\n".join(content)



    return sections

def scrape_visible_text(url, timeout=10):
    r = requests.get(url, timeout=timeout, headers={
        "User-Agent": "Mozilla/5.0"
    })
    r.raise_for_status()
    html = r.text
    # remove all HTML tags
    text = re.sub(r"<[^>]+>", " ", html)
    # collapse whitespace
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def scrape_all_paragraphs(url):
  headers = {"User-Agent": "Mozilla/5.0"}
  response = requests.get(url, headers=headers)
  response.raise_for_status()

  soup = BeautifulSoup(response.text, "html.parser")

  paragraphs = []
  for p in soup.find_all("p"):
      text = p.get_text(" ", strip=True)
      if text:
          paragraphs.append(text)

  return "\n".join(paragraphs)



In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from openai import OpenAI
import os
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)
coversation = ""
client = OpenAI(base_url="http://127.0.0.1:5000/v1", api_key="dummy")

@app.route("/api/respond", methods=["POST"])
def respond():
    data = request.json
    user_input = data.get("input", "")

    # initial context from KYM
    page_url, img_url = kym_search_image(user_input)
    sections = scrape_text_kym(page_url)

    context = ""
    for title, text in sections.items():
        context += f"{title}: {text}\n"

    # running text buffer
    combined = ""   # unlimited KYM text
    agentic = ""             # only new scraped material
    agentic_limit = 2500


    # loop until buffer fills
    while len(agentic) < agentic_limit:
      combined = context + "\n" + agentic

      # 1. ask model for search terms
      prompt = (
          "Given the following context, propose 3–5 broad search terms. "
          "Return ONLY a JSON array.\n\n"
          f"{combined}"
      )

      resp = client.chat.completions.create(
          model="Llama-Nemotron-Nano-4B-v1.1",
          messages=[
              {"role": "system", "content": "JSON only."},
              {"role": "user", "content": prompt}
          ],
          temperature=0.4,
          stream=False
      )

      import json
      try:
          terms = json.loads(resp.choices[0].message.content)
          if not isinstance(terms, list):
              break
      except:
          break

      # 2. scrape Google summaries for each term
      for term in terms:
          url = "https://www.google.com/search?q=" + term.replace(" ", "+")
          scraped = scrape_visible_text(url)
          if scraped:
              agentic += "\n" + scraped

          if len(agentic) >= agentic_limit:
              break

      # 3. compress ONLY the agentic portion
      compression_prompt = (
          "Summarize the following text. Maximum length 2500 characters.\n\n"
          f"{agentic[-3000:]}"
      )

      comp = client.chat.completions.create(
          model="Llama-Nemotron-Nano-4B-v1.1",
          messages=[
              {"role": "system", "content": "Summaries only."},
              {"role": "user", "content": compression_prompt}
          ],
          temperature=0.2,
          stream=False
      )

      agentic = comp.choices[0].message.content.strip()


    # final answer request
    final_prompt = (
        "Here is all the gathered context. Analyze the meme thoroughly.\n\n"
        f"{combined}"
    )
    print(final_prompt)
    final_resp = client.chat.completions.create(
        model="Llama-Nemotron-Nano-4B-v1.1",
        messages=[
            {"role": "system", "content": "Precise analysis."},
            {"role": "user", "content": final_prompt}
        ],
        temperature=0.6,
        stream=False
    )

    conversation += "This is the intial input: " + final_prompt + "\n";
    output = final_resp.choices[0].message.content
    conversation += "This is the intial output: " + output + "\n";
    return jsonify({"input": user_input, "output": output})

@app.route("/api/converse", methods=["POST"])
def converse():
  data = request.json
  user_input = data.get("input", "")
  prompt = (
    "Given the following context, give a response to the most recent input. Hopefully, 2-3 sentences. "
    "This is the most recent input (Focus on this): " + user_input + "\n"
    f"CONTEXT:\n{coversation}"
        )

  coversation += "This is added inputs by the user: " + user_input + "\n";


  messages = [
      {"role": "system", "content": "Be precise. Output JSON only."},
      {"role": "user", "content": prompt}
  ]

  resp = client.chat.completions.create(
      model="Llama-Nemotron-Nano-4B-v1.1",
      messages=messages,
      temperature=0.4,
      stream=False
  )

  output = resp.choices[0].message.content

  conversation += "This is the added output: " + output + "\n";

  return jsonify({"input": user_input, "output": output})

@app.route("/api/search_image", methods=["POST"])
def search_image():
    data = request.json
    query = data.get("query", "")

    if not query:
        return jsonify({"error": "query field required"}), 400

    img_url = kym_search_image(query)

    if img_url is None:
        return jsonify({"query": query, "image_url": None, "status": "not_found"})

    return jsonify({"query": query, "image_url": img_url})

NGROK_AUTH_TOKEN = "34oJGiQHYcc8THKsPzsqUXBLMi6_5oBmkk9dHcsN2T2S5W2gG"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(5001).public_url
print("Public URL:", public_url)

# -------------------
# Run server
# -------------------
app.run(port=5001)

Public URL: https://nonrealistic-ungrimed-luvenia.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit


In [41]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from openai import OpenAI
import os
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)
coversation = ""
client = OpenAI(base_url="http://127.0.0.1:5000/v1", api_key="dummy")

@app.route("/api/respond", methods=["POST"])
def respond():
    data = request.json
    user_input = data.get("input", "")

    # initial context from KYM
    page_url, img_url = kym_search_image(user_input)
    sections = scrape_text_kym(page_url)

    context = ""
    for title, text in sections.items():
        context += f"{title}: {text}\n"

    # running text buffer
    buffer = context.strip()
    max_chars = 2500 + len(context)

    # loop until buffer fills
    while len(buffer) < max_chars:
        # ask model for new search terms
        prompt = (
            "Given the following context, propose 3–5 broad google search terms "
            "that would expand the context. Return ONLY a JSON array of strings.\n\n"
            f"CONTEXT:\n{buffer[:2000]}"
        )

        messages = [
            {"role": "system", "content": "Be precise. Output JSON only."},
            {"role": "user", "content": prompt}
        ]

        resp = client.chat.completions.create(
            model="Llama-Nemotron-Nano-4B-v1.1",
            messages=messages,
            temperature=0.4,
            stream=False
        )

        raw_terms = resp.choices[0].message.content
        try:
            import json
            terms = json.loads(raw_terms)
            if not isinstance(terms, list):
                break
        except:
            break

        # scrape google summaries for each term
        for term in terms:
            google_page = f"https://www.google.com/search?q={term.replace(' ', '+')}"
            scraped = scrape_visible_text(google_page)
            if scraped:
                buffer += "\n" + scraped
                if len(buffer) >= max_chars:
                    break

        # compress via model
        compression_prompt = (
            "Summarize the important points of the following text. "
            "Be succinct and do not exceed 2 paragraphs.\n\n"
            f"{buffer[-3000:]}"
        )
        comp_resp = client.chat.completions.create(
            model="Llama-Nemotron-Nano-4B-v1.1",
            messages=[
                {"role": "system", "content": "Summaries only."},
                {"role": "user", "content": compression_prompt}
            ],
            temperature=0.3,
            stream=False
        )

        summary = comp_resp.choices[0].message.content.strip()
        buffer = summary  # next loop uses summary as new context

        if len(buffer) >= max_chars:
            break

    # final answer request
    final_prompt = (
        "Here is all the gathered context. Analyze the meme thoroughly.\n\n"
        f"{buffer[:max_chars]}"
    )

    final_resp = client.chat.completions.create(
        model="Llama-Nemotron-Nano-4B-v1.1",
        messages=[
            {"role": "system", "content": "Precise analysis."},
            {"role": "user", "content": final_prompt}
        ],
        temperature=0.6,
        stream=False
    )
    print(final_prompt)

    conversation += "This is the intial input: " + final_prompt + "\n";
    output = final_resp.choices[0].message.content
    conversation += "This is the intial output: " + output + "\n";

    return jsonify({"input": user_input, "output": output})

@app.route("/api/converse", methods=["POST"])
def converse():
  data = request.json
  user_input = data.get("input", "")
  prompt = (
    "Given the following context, give a response to the most recent input. Hopefully, 2-3 sentences. "
    "This is the most recent input (Focus on this): " + user_input + "\n"
    f"CONTEXT:\n{coversation}"
        )

  coversation += "This is added inputs by the user: " + user_input + "\n";


  messages = [
      {"role": "system", "content": "Be precise. Output JSON only."},
      {"role": "user", "content": prompt}
  ]

  resp = client.chat.completions.create(
      model="Llama-Nemotron-Nano-4B-v1.1",
      messages=messages,
      temperature=0.4,
      stream=False
  )

  output = resp.choices[0].message.content

  conversation += "This is the added output: " + output + "\n";

  return jsonify({"input": user_input, "output": output})

@app.route("/api/search_image", methods=["POST"])
def search_image():
    data = request.json
    query = data.get("query", "")

    if not query:
        return jsonify({"error": "query field required"}), 400

    img_url = kym_search_image(query)

    if img_url is None:
        return jsonify({"query": query, "image_url": None, "status": "not_found"})

    return jsonify({"query": query, "image_url": img_url})

NGROK_AUTH_TOKEN = "34oJGiQHYcc8THKsPzsqUXBLMi6_5oBmkk9dHcsN2T2S5W2gG"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(5001).public_url
print("Public URL:", public_url)

# -------------------
# Run server
# -------------------
app.run(port=5001)

Public URL: https://nonrealistic-ungrimed-luvenia.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Nov/2025 10:27:09] "OPTIONS /api/respond HTTP/1.1" 200 -


Page: https://knowyourmeme.com/memes/events/donald-trumps-grocery-id-gaffe
Image: https://i.kym-cdn.com/entries/icons/original/000/026/822/trump.jpg


INFO:werkzeug:127.0.0.1 - - [09/Nov/2025 10:27:51] "POST /api/respond HTTP/1.1" 200 -


Here is all the gathered context. Analyze the meme thoroughly.

Overview: Donald Trump's Grocery ID Gaffe refers to the reaction to a speech made by United States President Donald Trump , during which the president erroneously claimed that in order to purchase groceries, customers must have identification.
Background: On July 31st, 2018, at a rally in Tampa, Florida, President Trump delivered remarks in support of voter identification laws. During his speech, he compared voter ID laws to buying groceries, which he claimed people need identification to purchase. He said:
Development: Reactions to Trump's comments were split between people discussing their issues with voter ID laws and people mocking Trump for his assertion that people need identification to purchase groceries. Twitter  user @stuartpstevens tweeted, "We can’t get half of eligible American citizens to vote. Illegal voting is a felony and the idea that lots of people would be so hot to vote they’d commit a felony is one of

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

torch.cuda.empty_cache()
# Delete variables that might be holding onto GPU memory
# You might need to add other variable names here if you have more objects on the GPU
if 'model' in locals():
    del model
if 'pipe' in locals():
    del pipe
if 'pipe_smaller' in locals():
    del pipe_smaller
if 'input_ids' in locals():
    del input_ids
if 'outputs' in locals():
    del outputs
if 'tokenizer' in locals():
    del tokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("nvidia/NVIDIA-Nemotron-Nano-9B-v2")

# Load model (on GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(
    "nvidia/NVIDIA-Nemotron-Nano-9B-v2",
    torch_dtype=torch.bfloat16,  # saves memory, works on GPU
    trust_remote_code=True,
    device_map="auto"
)

# Example text input
input_text = "Complete the sentence. The quick brown fox jumps over the lazy "

# Tokenize input
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Generate output (limited to 50 tokens)
outputs = model.generate(**inputs, max_new_tokens=50)

# Decode output
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Input:", input_text)
print("Output:", decoded_output)


In [ ]:
!pip install flask flask-ngrok transformers torch -q
!pip install flask-ngrok

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Task
Replace `flask-ngrok` with `pyngrok` to expose the Flask application running in Google Colab via a public ngrok URL.

## Install pyngrok

### Subtask:
Add a cell to install the `pyngrok` library.


**Reasoning**:
The subtask is to install the `pyngrok` library. This can be done using pip in a new code cell.



In [ ]:
!pip install pyngrok

## Modify flask app

### Subtask:
Update the Flask application cell to remove `flask-ngrok` and integrate `pyngrok` to establish the tunnel.


**Reasoning**:
Update the Flask application code to use `pyngrok` instead of `flask-ngrok` to expose the application. This involves removing the old import and function call, importing `ngrok` from `pyngrok`, connecting to the default Flask port (5000) using `ngrok.connect()`, and printing the public URL.



**Reasoning**:
The previous attempt to connect with ngrok failed because it requires authentication. To resolve this, I need to add the ngrok authtoken before connecting. I will use the `ngrok.set_auth_token()` function for this purpose.



## Run the flask app with pyngrok

### Subtask:
Execute the modified Flask app cell to start the server and the ngrok tunnel using pyngrok.


**Reasoning**:
Execute the modified Flask app cell to start the server and the ngrok tunnel. The instructions indicate to run the cell containing the modified Flask application.



## Run the flask app with pyngrok

### Subtask:
Execute the modified Flask app cell to start the server and the ngrok tunnel using pyngrok.


**Reasoning**:
Execute the cell containing the modified Flask application code to start the server and the ngrok tunnel using pyngrok.



In [ ]:
!pip install flask pyngrok

In [ ]:
!pip install flask-cors

In [ ]:
# Install dependencies
!pip install flask flask-cors pyngrok transformers torch --quiet

from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

app = Flask(__name__)
CORS(app)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer
print("Loading Nemotron-Nano-9B-v2...")
tokenizer = AutoTokenizer.from_pretrained("nvidia/NVIDIA-Nemotron-Nano-9B-v2", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "nvidia/NVIDIA-Nemotron-Nano-9B-v2",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
model.eval()
print("Model loaded!")

# -------------------
# Function: Two-step reasoning generation
# -------------------
def generate_with_reasoning(user_input, max_thinking_tokens=32, max_total_tokens=128, temperature=0.7, top_p=0.9):
    """
    Approximates Hugging Face's ThinkingBudgetClient:
    1. Generate reasoning trace first
    2. Generate final answer conditioned on reasoning
    """

    # 1. Generate reasoning content (short)
    reasoning_prompt = f"You are a helpful AI assistant. /think\nUser: {user_input}\nAssistant:"

    reasoning_inputs = tokenizer(reasoning_prompt, return_tensors="pt").to(device)
    reasoning_outputs = model.generate(
        **reasoning_inputs,
        max_new_tokens=max_thinking_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
    )
    reasoning_text = tokenizer.decode(reasoning_outputs[0], skip_special_tokens=True)

    # Ensure reasoning text ends cleanly with </think>
    if "</think>" not in reasoning_text:
        reasoning_text += "\n</think>"

    # 2. Generate final answer conditioned on reasoning
    final_prompt = f"{reasoning_text}\nUser: {user_input}\nAssistant:"
    final_inputs = tokenizer(final_prompt, return_tensors="pt").to(device)
    remaining_tokens = max_total_tokens - final_inputs.input_ids.shape[1]

    final_outputs = model.generate(
        **final_inputs,
        max_new_tokens=remaining_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
    )
    final_text = tokenizer.decode(final_outputs[0], skip_special_tokens=True)

    # Strip out the reasoning trace from final output
    # Keep only text after </think> or the last 'Assistant:' line
    if "</think>" in final_text:
        final_text = final_text.split("</think>")[-1].strip()
    elif "Assistant:" in final_text:
        final_text = final_text.split("Assistant:")[-1].strip()

    return final_text

# -------------------
# Flask endpoint
# -------------------
@app.route("/api/respond", methods=["POST"])
def respond():
    data = request.json
    user_input = data.get("input", "")

    output_text = generate_with_reasoning(user_input)
    return jsonify({"input": user_input, "output": output_text})

# -------------------
# ngrok tunnel
# -------------------
NGROK_AUTH_TOKEN = "34oJGiQHYcc8THKsPzsqUXBLMi6_5oBmkk9dHcsN2T2S5W2gG"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(5000).public_url
print("Public URL:", public_url)

# -------------------
# Run server
# -------------------
app.run(port=5000)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Clear the CUDA cache
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())
# Delete variables that might be holding onto GPU memory
# You might need to add other variable names here if you have more objects on the GPU
if 'model' in locals():
    del model
if 'pipe' in locals():
    del pipe
if 'pipe_smaller' in locals():
    del pipe_smaller
if 'input_ids' in locals():
    del input_ids
if 'outputs' in locals():
    del outputs
if 'tokenizer' in locals():
    del tokenizer

# Run Python's garbage collector
import gc
gc.collect()

print("GPU memory hopefully cleared.")

# Choose device (this variable is not used in pipeline after fixing the error)
device = 0 if torch.cuda.is_available() else -1

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(
    "nvidia/Llama-3.1-Nemotron-Nano-4B-v1.1",
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    "nvidia/Llama-3.1-Nemotron-Nano-4B-v1.1",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.eval()

# Create a pipeline for simplicity
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # Removed the device=device argument as recommended by the error
)

# Function to complete a sentence
def complete_sentence(prompt: str, max_new_tokens: int = 50) -> str:
    outputs = generator(
    prompt,
    max_new_tokens=50,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
      repetition_penalty=1.15,
      pad_token_id=tokenizer.eos_token_id,
    )

    # The pipeline output format might differ slightly from model.generate
    # Adjust decoding if necessary based on pipeline's return
    return outputs[0]["generated_text"]

# Example usage
prompt = "The quick brown fox jumped over the"
result = complete_sentence(prompt, max_new_tokens=10)
print("Result:", result)

In [ ]:
# Clear the CUDA cache
torch.cuda.empty_cache()

# Delete variables that might be holding onto GPU memory
# You might need to add other variable names here if you have more objects on the GPU
if 'model' in locals():
    del model
if 'pipe' in locals():
    del pipe
if 'pipe_smaller' in locals():
    del pipe_smaller
if 'input_ids' in locals():
    del input_ids
if 'outputs' in locals():
    del outputs
if 'tokenizer' in locals():
    del tokenizer

# Run Python's garbage collector
import gc
gc.collect()

print("GPU memory hopefully cleared.")

## Summary:

## Data Analysis Key Findings

*   The `pyngrok` library was successfully installed.
*   Attempts to establish an ngrok tunnel using `pyngrok` failed due to an authentication error (`ERR_NGROK_4018`).
*   The authentication failure occurred because the `NGROK_AUTH_TOKEN` environment variable was not set in the Google Colab environment.
*   The Flask application server did not start successfully as the script terminated after the `ngrok.connect()` call failed.

## Insights or Next Steps

*   To successfully expose the Flask application via a public ngrok URL, the user must set the `NGROK_AUTH_TOKEN` environment variable in their Google Colab environment or secrets with a valid ngrok authentication token.
*   After setting the authtoken, re-executing the Flask application cell is required to attempt establishing the ngrok tunnel and starting the Flask server.
